# Module 1: Course Introduction & Local Setup
## Lesson 5: First API Call & System Prompts

### 📄 Overview
In this lesson, we graduate from local setup to writing actual code that interacts with the OpenAI API. We learn the strict JSON structure required by the Chat Completions API and explore how to control the model's behavior using **System Prompts**.

### 🗝️ Key Concepts
* **Chat Completions API**: The standard endpoint (`chat.completions.create`) used by almost all modern LLM providers.
* **Message Roles**:
    * **`system`**: Sets the behavior, tone, and rules (The "God Mode" instruction).
    * **`user`**: The actual input or question from the end-user.
    * **`assistant`**: The response generated by the model (used for history).
* **Statelessness**: The API does not remember you. Every time you call it, you must send the entire conversation history if you want context.

### 🛠️ Technical Implementation
The core object is a **List of Dictionaries**, where each dictionary represents a message.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# 1. Load Environment Variables
# This looks for the .env file we created in Lesson 4
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    print("❌ Error: API Key not found!")
else:
    print("✅ API Key loaded.")

# 2. Initialize the Client
# This client handles the connection pooling and authentication for us
client = OpenAI(api_key=api_key)

In [ ]:
# 1. Define the conversation
messages = [
    {
        "role": "user", 
        "content": "Hello GPT, this is my first ever message to you via Python code!"
    }
]

# 2. Make the API Call
# We use 'gpt-4o-mini' which is the current cost-effective standard
print("Sending request to OpenAI...")
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

# 3. Extract and Print the content
# The response object contains metadata (usage, id, etc.) + the choices
ai_message = response.choices[0].message.content
print("\n--- AI Response ---")
print(ai_message)

In [ ]:
def ask_gpt(user_query, system_behavior):
    """
    A helper function to test different system personas.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_behavior},
            {"role": "user", "content": user_query}
        ]
    )
    return response.choices[0].message.content

# Test 1: Helpful Assistant
print("--- 😇 Helpful ---")
print(ask_gpt("What is 2 + 2?", "You are a helpful math tutor."))

# Test 2: Snarky Assistant
print("\n--- 😈 Snarky ---")
print(ask_gpt("What is 2 + 2?", "You are a snarky, sarcastic assistant who hates answering simple questions."))

### 🧪 Lab Notes & Engineering Log

*The following experiments focus on Persona Injection and API Inspection.*

#### Experiment 1: Persona Stress Test
**Objective:** See how strictly the model adheres to the system prompt.
**Test:**
* **System Prompt:** *"You are a Cowboy from 1850. You do not know what computers are."*
* **User Prompt:** *"Help me fix my Python bug."*

#### Experiment 2: Inspecting the Full Response Object
**Objective:** As engineers, we need to know how much this cost.
**Code Modification:**
```python
full_response = client.chat.completions.create(model="gpt-4o-mini", messages=[...])
print(full_response.usage)